# コンペの目標
乗客の生存予測

**データの読み込み**

In [22]:
# csvファイルを読み込んで内容を確認
import pandas as pd
import numpy as np

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
gender_submission = pd.read_csv("data/gender_submission.csv")

**特徴量エンジニアリング**

In [28]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [29]:
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
dtype: int64

In [30]:
# train, testをくっつけておくといいことがある
data = pd.concat([train, test], sort=False) 
# カテゴリカル変数を量的変数に変換
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna('S', inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
age_avg = data['Age'].mean()
age_std = data['Age'].std()
# Ageの正規分布の中の中間付近の値から補完値を決定している
data['Age'].fillna(np.random.randint(age_avg-age_std, age_avg+age_std), inplace=True)

# 直接使用しない列を削除
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

In [31]:
# 学習のためにデータを分割して元に戻す
train = data[:len(train)]
test = data[len(train):]

In [32]:
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop(['Survived'], axis=1)

機械学習モデルを作成

In [35]:
# モデルをインポート
from sklearn.linear_model import LogisticRegression

# モデルを作成
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
# データを入れて学習
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


/Users/gotokenichiro/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**submitの作成**

In [37]:
sub = pd.read_csv('data/gender_submission.csv')
sub['Survived'] = list(map(int, y_pred))
# csvファイルを作成
sub.to_csv('submission.csv', index=False)

In [2]:
import pandas as pd
import pandas_profiling

train = pd.read_csv('data/test.csv')
train.profile_report()

/Users/gotokenichiro/anaconda3/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/gotokenichiro/anaconda3/lib/python3.11/site-packages/visions/backends/shared/nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.6/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.6/u/import-error